In [ ]:
from sklearn import metrics

def getNeighborMedian(nbr):
    if nbr == 'MeadowV':
        return 88000 
    elif nbr == 'IDOTRR':
        return 103000
    elif nbr == 'BrDale':
        return 106000  
    elif nbr == 'OldTown':
        return 119000   
    elif nbr == 'Edwards':
        return 121750
    elif nbr == 'BrkSide':
        return 124300
    elif nbr == 'Sawyer':
        return 135000
    elif nbr == 'Blueste':
        return 137500
    elif nbr == 'SWISU':
        return 139500
    elif nbr == 'NAmes':
        return 140000
    elif nbr == 'NPkVill':
        return 146000
    elif nbr == 'Mitchel':
        return 153500
    elif nbr == 'SawyerW':
        return 179900    
    elif nbr == 'Gilbert':
        return 181000   
    elif nbr == 'NWAmes':
        return 182900    
    elif nbr == 'Blmngtn':
        return 191000   
    elif nbr == 'CollgCr':
        return 197200
    elif nbr == 'ClearCr':
        return 200250
    elif nbr == 'Crawfor':
        return 200624
    elif nbr == 'Veenker':
        return 218000
    elif nbr == 'Somerst':
        return 225500
    elif nbr == 'Timber':
        return 228475
    elif nbr == 'StoneBr':
        return 278000  
    elif nbr == 'NoRidge':
        return 301500   
    elif nbr == 'NridgHt':
        return 315000  
    
def rmsle(y_true,y_pred):
   assert len(y_true) == len(y_pred)
   return np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean() ** 0.5
    

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

%matplotlib inline
#panda is a read/write tool

train_file = "c:\\house\\train.csv"
test_file = "c:\\house\\test.csv"

# comma delimited is the default
hsData = pd.read_csv(train_file, header = 0)
hsData = hsData.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'LotFrontage', 'FireplaceQu'], axis=1)

# The 'MSZoneClass_Composite' could cause noise
#hsData['MSZoneClass_Composite'] = hsData['MSZoning'] + hsData['MSSubClass']

# The composite 'Total_FlrSF' improves some
hsData['Total_FlrSF'] = hsData['1stFlrSF'] + hsData['2ndFlrSF']

# The composite 'Total_Bath' improves a bit
hsData['Total_Bath'] = hsData['FullBath'] + hsData['BsmtFullBath'] + hsData['HalfBath']*0.5 + hsData['BsmtHalfBath']*0.5

# The composite 'Garage_Composite', 'Exter_Composite' and 'Bsmt_Composite' improves a lot
hsData['GarageQual'] = hsData['GarageQual'].fillna('NA')
hsData['GarageCond'] = hsData['GarageCond'].fillna('NA')
hsData['Garage_Composite'] = hsData['GarageQual'] + hsData['GarageCond']
hsData['Garage_Composite'].replace(['PoPo','NANA','FaFa','FaPo','FaTA','ExEx','TAFa','TAGd', \
                             'TATA','GdTA','GdGd','ExTA'], \
                             [97,100,110,116,116,125,125,144,171,209,227,475], inplace=True)

hsData['Exter_Composite'] = hsData['ExterQual'] + hsData['ExterCond']
hsData['Exter_Composite'].replace(['FaFa','TAPo','FaTA','TAFa','TAEx','TATA','TAGd','GdGd', \
                             'GdTA','ExGd','ExEx','ExTA'], \
                             [61,77,99,119,140,140,140,210,221,267,325,374], inplace=True)

hsData['BsmtQual'] = hsData['BsmtQual'].fillna('NA')
hsData['BsmtCond'] = hsData['BsmtCond'].fillna('NA')
hsData['Bsmt_Composite'] = hsData['BsmtQual'] + hsData['BsmtCond']
hsData['Bsmt_Composite'].replace(['FaPo','NANA','FaTA','FaFa','TAFa','TATA','TAGd','GdFa', \
                            'GdTA','GdGd','ExGd','ExTA'], \
                             [64,112,114,114,114,135,164,182,192,199,275,318], inplace=True)

# Trim below noise improves a bit
hsData = hsData[hsData["GrLivArea"] < 3500]
hsData = hsData[hsData["GarageArea"] < 1200]
hsData = hsData[hsData["Total_FlrSF"] < 3500]
hsData = hsData[hsData["Total_FlrSF"] > 500]
hsData.info()
#hsData.describe()

In [ ]:
num_train_features = hsData.select_dtypes(include=[np.number])
num_train_features.dtypes

num_train_features = num_train_features.drop(['MasVnrArea', 'GarageYrBlt'], axis=1)
num_train_features['Neighborhood'] = hsData['Neighborhood'].apply(getNeighborMedian)

corr = num_train_features.corr()
print (corr['SalePrice'].sort_values(ascending=False)[:35], '\n')

# Drop 'Id', 'SalePrice' after evaluation
num_train_features = num_train_features.drop(['Id', 'SalePrice'], axis=1)

#num_train_features.info()

In [ ]:
hsTest = pd.read_csv(test_file, header = 0)
hsTest = hsTest.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'LotFrontage', 'FireplaceQu'], axis=1)

hsTest['Total_FlrSF'] = hsTest['1stFlrSF'] + hsTest['2ndFlrSF']

hsTest['FullBath'] = hsTest['FullBath'].fillna(0)
hsTest['BsmtFullBath'] = hsTest['BsmtFullBath'].fillna(0)
hsTest['HalfBath'] = hsTest['HalfBath'].fillna(0)
hsTest['BsmtHalfBath'] = hsTest['BsmtHalfBath'].fillna(0)
hsTest['Total_Bath'] = hsTest['FullBath'] + hsTest['BsmtFullBath'] + hsTest['HalfBath']*0.5 + hsTest['BsmtHalfBath']*0.5

hsTest['GarageQual'] = hsTest['GarageQual'].fillna('NA')
hsTest['GarageCond'] = hsTest['GarageCond'].fillna('NA')
hsTest['Garage_Composite'] = hsTest['GarageQual'] + hsTest['GarageCond']
hsTest['Garage_Composite'].replace(['PoPo','PoFa','NANA','FaFa','NAFa','FaPo','FaTA','ExEx','TAFa','TAPo','TAGd', \
                             'TATA','NATA','GdTA','GdGd','GdEx','ExTA'], \
                             [97,100,100,110,110,116,116,125,125,110,144,171,171,209,227,250,475], inplace=True)

hsTest['ExterQual'] = hsTest['ExterQual'].fillna('NA')
hsTest['ExterCond'] = hsTest['ExterCond'].fillna('NA')
hsTest['Exter_Composite'] = hsTest['ExterQual'] + hsTest['ExterCond']
hsTest['Exter_Composite'].replace(['FaPo','FaFa','FaGd','TAPo','FaTA','TAFa','TAEx','TATA','TAGd','GdGd', \
                             'GdFa','GdTA','GdEx','ExGd', 'ExEx', 'ExTA'], \
                             [50,61,70,77,99,119,140,140,140,210,210,221,230,267,325,374], inplace=True)

hsTest['BsmtQual'] = hsTest['BsmtQual'].fillna('NA')
hsTest['BsmtCond'] = hsTest['BsmtCond'].fillna('NA')
hsTest['Bsmt_Composite'] = hsTest['BsmtQual'] + hsTest['BsmtCond']
hsTest['Bsmt_Composite'].replace(['FaPo','FaGd','NANA','FaTA','FaFa','NAFa','TAPo','TAFa','TATA','TANA','NATA','TAGd','GdFa', \
                            'GdTA','GdGd','GdNA','ExPo','ExGd','ExTA'], \
                             [64,80,112,114,114,114,114,114,135,135,135,164,182,192,199,199,260,275,318], inplace=True)

#hsTest.info()
#hsTest.describe()

In [ ]:
num_test_features = hsTest.select_dtypes(include=[np.number])
num_test_features.dtypes

num_test_features = num_test_features.drop(['Id', 'MasVnrArea', 'GarageYrBlt'], axis=1)

num_test_features['GarageCars'] = num_test_features['GarageCars'].fillna(0)
num_test_features['GarageArea'] = num_test_features['GarageArea'].fillna(0)
num_test_features['BsmtFinSF1'] = num_test_features['BsmtFinSF1'].fillna(0)
num_test_features['BsmtFinSF2'] = num_test_features['BsmtFinSF2'].fillna(0)
num_test_features['BsmtUnfSF'] = num_test_features['BsmtUnfSF'].fillna(0)
num_test_features['TotalBsmtSF'] = num_test_features['TotalBsmtSF'].fillna(0)
num_test_features['BsmtFullBath'] = num_test_features['BsmtFullBath'].fillna(0)
num_test_features['BsmtHalfBath'] = num_test_features['BsmtHalfBath'].fillna(0)
num_test_features['Neighborhood'] = hsTest['Neighborhood'].apply(getNeighborMedian)

#num_test_features.info()
num_test_features.isnull().values.any()

null_columns=num_test_features.columns[num_test_features.isnull().any()]
num_test_features[null_columns].isnull().sum()


In [ ]:
# Lasso Regressor
from sklearn.linear_model import Lasso
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

ls = Lasso(alpha=0.00055)
ls.fit(X_train, y_train)

X_test_predict = ls.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(ls)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Hs_Test = num_test_features
Hs_test_predict = ls.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Lasso Regressor')
plt.show()

submit_file = 'c:\\house\\submission_ls.csv'
print submit_file
submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": Hs_test_predict
    })

submission.to_csv(submit_file, index=False)

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict) + 0.04705
print("Kaggle Possible: %s\n" % kg)

In [ ]:
# XG boost, by 'conda install py-xgboost'
from xgboost import XGBRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

xg = XGBRegressor(max_depth=5, n_estimators=700)

xg.fit(X_train, y_train)

X_test_predict = xg.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(xg)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Hs_Test = num_test_features
Hs_test_predict = xg.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('XG Boost')
plt.show()

submit_file = 'c:\\house\\submission_xg.csv'
print submit_file
submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": Hs_test_predict
    })

submission.to_csv(submit_file, index=False)

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict) + 0.04705
print("Kaggle Possible: %s\n" % kg)

In [ ]:
# Bagging Regressor
from sklearn.ensemble import BaggingRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

br = BaggingRegressor(n_estimators=15)
br.fit(X_train, y_train)

X_test_predict = br.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(br)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Bagging Regressor')
plt.show()

In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

rf = RandomForestRegressor(n_estimators = 150, max_depth=5)
rf.fit(X_train, y_train)

X_test_predict = rf.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(rf)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Random Forest Regressor')
plt.show()

In [ ]:
# Epsilon-Support Vector Regressor
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

svr = SVR(gamma=0.001, C=100, kernel='rbf')
svr.fit(X_train, y_train)

X_test_predict = svr.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(svr)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('SVR Regressor')
plt.show()
